In [ ]:
import os
execfile("../script/utils.py")
eventsPath = os.environ["YAHOO_DATA"]
splitedRdd = sc.textFile(eventsPath + "/splitedData")
splitedRdd = splitedRdd.map(parseContextData2)
#splitedRdd.take(1)
#splitedRdd = sc.parallelize(splitedRdd.take(2000))
splitedRdd.take(1)
#(uid,[[train],[test]]

In [ ]:
numRecList = 5
def topFiveSortedList(oldList, context, probability):
    for x in range(0,len(oldList)-1):
        if x == 0 and oldList[x][1] < probability:
            oldList[x] = (context,probability)
            if oldList[x+1][1] < oldList[x][1]:
                temp = oldList[x+1] 
                oldList[x+1] = oldList[x]
                oldList[x] = temp 
            else:
                break
        elif oldList[x+1][1] < oldList[x][1]:
            temp = oldList[x+1] 
            oldList[x+1] = oldList[x]
            oldList[x] = temp 
        else:
            break
    #return sorted(oldList,key=lambda x: -x[1])
    return oldList      
def evaluation_rr(testList, recList):
    scores = 0
    for i in range(len(testList)):
        for j in range(len(recList[0])):
            if testList[i] == recList[i][j]:
                scores = scores + 1.0 / (j+1)
                break
    scores = scores /  len(testList)
    return scores
def evaluation_ap(testList, recList):
    numRec = len(recList[0])
    scores = 0
    num = 0 
    rr = []
    rr[:len(recList)] = [0 for i in range(numRec)]

    for i in range(len(testList)):
        for j in range(len(recList[0])):
            if testList[i] == recList[i][j]:
                rr[j] = 1
                break
    
    for i in range(len(testList)):
        for j in range(numRec):
            if testList[i] == recList[i][j]:
                for k in range(j,numRec):
                    if rr[k] == 1:
                        scores = scores + 1.0 / (k+1)
                        num = num + 1
                break
    if num == 0:
        scores = 0
    else:
        scores = scores / num
    return scores
def evaluation_recall(testList, recList):
    numRec = len(recList[0])
    scores = 0
    num = 0
    recall = []
    recall[:len(recList)] = [0 for i in range(numRec)]
    rr=[]
    rr[:len(recList)] = [0 for i in range(numRec)]
    for i in range(len(testList)):
        for j in range(len(recList[0])):
            if testList[i] == recList[i][j]:
                rr[j] = 1
                break
    for i in range(len(testList)):
        for j in range(numRec):
            if testList[i] == recList[i][j]:
                for k in range(j,numRec):
                        recall[k] = recall[k] + 1.0
                        num = num + 1
                break 
    
    recall = [0 if num == 0 else x*sum(rr)/num for x in recall]
    return recall
def remove_duplicates(values):
    output = []
    seen = set()
    for value in values:
        # If value has not been encountered yet,
        # ... add it to both list and set.
        if value not in seen:
            output.append(value)
            seen.add(value)
    return output

def print_result(finalScore, algorithm_name):
    f = open('result/resultOfEvaluation.txt','a')
    f.write(algorithm_name + ' by mrr:'+str(finalScore.map(lambda x:x[0]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by map:'+str(finalScore.map(lambda x:x[1]).mean()))
    f.write('\n')
    f.write(algorithm_name + ' by recall:')
    for i in range(numRecList):
        if i == 0:
            f.write(' '+str(finalScore.map(lambda x:x[2][i]).mean()))
        elif i == numRecList-1:
            f.write(','+str(finalScore.map(lambda x:x[2][i]).mean()) + '\n')
        else:
            f.write(','+str(finalScore.map(lambda x:x[2][i]).mean()))
    f.close()

In [ ]:
def bayesian_context_moving(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    trainList = [(t[0],t[1][4]) for t in trainList] #itemId,movingId
    testList = [(t[0],t[1][4]) for t in testList]   #itemId,movingID
    numTrain = len(trainList)
    numTest = l - numTrain

    newTestList = []
    for t in testList:
        context = [x for x in trainList if x[1]==t[1]]
        numContext = float(len(context))
        if numTrain != 0:
            p_context = numContext/numTrain  #P(C1i, C2j, C3k)
        else:
            p_context = 0
        p_app = [(-1,0),(-1,0),(-1,0),(-1,0),(-1,0)]
        context_no_duplicate = remove_duplicates(context)
        for c in context_no_duplicate:
            appi = [x for x in trainList if x[0]==c[0]]
            numAppi = float(len(appi))
            if numTrain != 0:
                p_appi = numAppi/numTrain
            else:
                p_appi = 0
            contextAppi = [x for x in trainList if x[0]==c[0] and x[1]==c[1]]
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                p_contextAppi = len(contextAppi)/numAppi 
            else:
                p_contextAppi = 0
            if p_context != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p = p_contextAppi * p_appi / p_context
            else:
                p = 0
            p_app = topFiveSortedList(p_app,c[0],p)
        p_app = sorted(p_app,key=lambda x: -x[1])
        app_rec = map(lambda x:x[0],p_app[:5])
        newTestList.append((t[0],app_rec))
    testList = [x[0] for x in newTestList]
    finalRecommender = [x[1] for x in newTestList]
    scores_rr = evaluation_rr(testList, finalRecommender)
    scores_ap = evaluation_ap(testList, finalRecommender)
    scores_recall = evaluation_recall(testList, finalRecommender)
    return scores_rr, scores_ap, scores_recall
finalScore = splitedRdd.map(bayesian_context_moving)
print_result(finalScore, 'bayesian_context_moving')

In [ ]:
def bayesian_context_location(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    trainList = [(t[0],t[1][5]) for t in trainList] #itemId,movingId
    testList = [(t[0],t[1][5]) for t in testList]   #itemId,movingID
    numTrain = len(trainList)
    numTest = l - numTrain

    newTestList = []
    for t in testList:
        context = [x for x in trainList if x[1]==t[1]]
        numContext = float(len(context))
        if numTrain != 0:
            p_context = numContext/numTrain  #P(C1i, C2j, C3k)
        else:
            p_context = 0
        p_app = [(-1,0),(-1,0),(-1,0),(-1,0),(-1,0)]
        context_no_duplicate = remove_duplicates(context)
        for c in context_no_duplicate:
            appi = [x for x in trainList if x[0]==c[0]]
            numAppi = float(len(appi))
            if numTrain != 0:
                p_appi = numAppi/numTrain
            else:
                p_appi = 0
            contextAppi = [x for x in trainList if x[0]==c[0] and x[1]==c[1]]
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                p_contextAppi = len(contextAppi)/numAppi 
            else:
                p_contextAppi = 0
            if p_context != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p = p_contextAppi * p_appi / p_context
            else:
                p = 0
            p_app = topFiveSortedList(p_app,c[0],p)
        p_app = sorted(p_app,key=lambda x: -x[1])
        app_rec = map(lambda x:x[0],p_app[:5])
        newTestList.append((t[0],app_rec))
    testList = [x[0] for x in newTestList]
    finalRecommender = [x[1] for x in newTestList]
    scores_rr = evaluation_rr(testList, finalRecommender)
    scores_ap = evaluation_ap(testList, finalRecommender)
    scores_recall = evaluation_recall(testList, finalRecommender)
    return scores_rr, scores_ap, scores_recall
finalScore = splitedRdd.map(bayesian_context_location)
print_result(finalScore, 'bayesian_context_location')

In [ ]:
def bayesian_context_time(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    trainList = [(t[0],t[1][6]) for t in trainList] #itemId,movingId
    testList = [(t[0],t[1][6]) for t in testList]   #itemId,movingID
    numTrain = len(trainList)
    numTest = l - numTrain

    newTestList = []
    for t in testList:
        context = [x for x in trainList if x[1]==t[1]]
        numContext = float(len(context))
        if numTrain != 0:
            p_context = numContext/numTrain  #P(C1i, C2j, C3k)
        else:
            p_context = 0
        p_app = [(-1,0),(-1,0),(-1,0),(-1,0),(-1,0)]
        context_no_duplicate = remove_duplicates(context)
        for c in context_no_duplicate:
            appi = [x for x in trainList if x[0]==c[0]]
            numAppi = float(len(appi))
            if numTrain != 0:
                p_appi = numAppi/numTrain
            else:
                p_appi = 0
            contextAppi = [x for x in trainList if x[0]==c[0] and x[1]==c[1]]
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                p_contextAppi = len(contextAppi)/numAppi 
            else:
                p_contextAppi = 0
            if p_context != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p = p_contextAppi * p_appi / p_context
            else:
                p = 0
            p_app = topFiveSortedList(p_app,c[0],p)
        p_app = sorted(p_app,key=lambda x: -x[1])
        app_rec = map(lambda x:x[0],p_app[:5])
        newTestList.append((t[0],app_rec))
    testList = [x[0] for x in newTestList]
    finalRecommender = [x[1] for x in newTestList]
    scores_rr = evaluation_rr(testList, finalRecommender)
    scores_ap = evaluation_ap(testList, finalRecommender)
    scores_recall = evaluation_recall(testList, finalRecommender)
    return scores_rr, scores_ap, scores_recall
finalScore = splitedRdd.map(bayesian_context_time)
print_result(finalScore, 'bayesian_context_time')

In [ ]:
def bayesian_context_c12(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    trainList = [(t[0],t[1][4],t[1][5]) for t in trainList] #itemId,movingId
    testList = [(t[0],t[1][4],t[1][5]) for t in testList]   #itemId,movingID
    numTrain = len(trainList)
    numTest = l - numTrain

    newTestList = []
    for t in testList:
        context = [x for x in trainList if x[1]==t[1] and x[2]==t[2]]
        numContext = float(len(context))
        if numTrain != 0:
            p_context = numContext/numTrain  #P(C1i, C2j, C3k)
        else:
            p_context = 0
        p_app = [(-1,0),(-1,0),(-1,0),(-1,0),(-1,0)]
        context_no_duplicate = remove_duplicates(context)
        for c in context_no_duplicate:
            appi = [x for x in trainList if x[0]==c[0]]
            numAppi = float(len(appi))
            if numTrain != 0:
                p_appi = numAppi/numTrain
            else:
                p_appi = 0
            contextAppi = [x for x in trainList if x[0]==c[0] and x[1]==c[1] and x[2]==c[2]]
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                p_contextAppi = len(contextAppi)/numAppi 
            else:
                p_contextAppi = 0
            if p_context != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p = p_contextAppi * p_appi / p_context
            else:
                p = 0
            p_app = topFiveSortedList(p_app,c[0],p)
        p_app = sorted(p_app,key=lambda x: -x[1])
        app_rec = map(lambda x:x[0],p_app[:5])
        newTestList.append((t[0],app_rec))
    testList = [x[0] for x in newTestList]
    finalRecommender = [x[1] for x in newTestList]
    scores_rr = evaluation_rr(testList, finalRecommender)
    scores_ap = evaluation_ap(testList, finalRecommender)
    scores_recall = evaluation_recall(testList, finalRecommender)
    return scores_rr, scores_ap, scores_recall
finalScore = splitedRdd.map(bayesian_context_c12)
print_result(finalScore, 'bayesian_context_c12')

In [ ]:
def bayesian_context_c13(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    trainList = [(t[0],t[1][4],t[1][6]) for t in trainList] #itemId,movingId
    testList = [(t[0],t[1][4],t[1][6]) for t in testList]   #itemId,movingID
    numTrain = len(trainList)
    numTest = l - numTrain

    newTestList = []
    for t in testList:
        context = [x for x in trainList if x[1]==t[1] and x[2]==t[2]]
        numContext = float(len(context))
        if numTrain != 0:
            p_context = numContext/numTrain  #P(C1i, C2j, C3k)
        else:
            p_context = 0
        p_app = [(-1,0),(-1,0),(-1,0),(-1,0),(-1,0)]
        context_no_duplicate = remove_duplicates(context)
        for c in context_no_duplicate:
            appi = [x for x in trainList if x[0]==c[0]]
            numAppi = float(len(appi))
            if numTrain != 0:
                p_appi = numAppi/numTrain
            else:
                p_appi = 0
            contextAppi = [x for x in trainList if x[0]==c[0] and x[1]==c[1] and x[2]==c[2]]
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                p_contextAppi = len(contextAppi)/numAppi 
            else:
                p_contextAppi = 0
            if p_context != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p = p_contextAppi * p_appi / p_context
            else:
                p = 0
            p_app = topFiveSortedList(p_app,c[0],p)
        p_app = sorted(p_app,key=lambda x: -x[1])
        app_rec = map(lambda x:x[0],p_app[:5])
        newTestList.append((t[0],app_rec))
    testList = [x[0] for x in newTestList]
    finalRecommender = [x[1] for x in newTestList]
    scores_rr = evaluation_rr(testList, finalRecommender)
    scores_ap = evaluation_ap(testList, finalRecommender)
    scores_recall = evaluation_recall(testList, finalRecommender)
    return scores_rr, scores_ap, scores_recall
finalScore = splitedRdd.map(bayesian_context_c13)
print_result(finalScore, 'bayesian_context_c13')

In [ ]:
def bayesian_context_c23(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    trainList = [(t[0],t[1][5],t[1][6]) for t in trainList] #itemId,movingId
    testList = [(t[0],t[1][5],t[1][6]) for t in testList]   #itemId,movingID
    numTrain = len(trainList)
    numTest = l - numTrain

    newTestList = []
    for t in testList:
        context = [x for x in trainList if x[1]==t[1] and x[2]==t[2]]
        numContext = float(len(context))
        if numTrain != 0:
            p_context = numContext/numTrain  #P(C1i, C2j, C3k)
        else:
            p_context = 0
        p_app = [(-1,0),(-1,0),(-1,0),(-1,0),(-1,0)]
        context_no_duplicate = remove_duplicates(context)
        for c in context_no_duplicate:
            appi = [x for x in trainList if x[0]==c[0]]
            numAppi = float(len(appi))
            if numTrain != 0:
                p_appi = numAppi/numTrain
            else:
                p_appi = 0
            contextAppi = [x for x in trainList if x[0]==c[0] and x[1]==c[1] and x[2]==c[2]]
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                p_contextAppi = len(contextAppi)/numAppi 
            else:
                p_contextAppi = 0
            if p_context != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p = p_contextAppi * p_appi / p_context
            else:
                p = 0
            p_app = topFiveSortedList(p_app,c[0],p)
        p_app = sorted(p_app,key=lambda x: -x[1])
        app_rec = map(lambda x:x[0],p_app[:5])
        newTestList.append((t[0],app_rec))
    testList = [x[0] for x in newTestList]
    finalRecommender = [x[1] for x in newTestList]
    scores_rr = evaluation_rr(testList, finalRecommender)
    scores_ap = evaluation_ap(testList, finalRecommender)
    scores_recall = evaluation_recall(testList, finalRecommender)
    return scores_rr, scores_ap, scores_recall
finalScore = splitedRdd.map(bayesian_context_c23)
print_result(finalScore, 'bayesian_context_c23')

In [ ]:
def bayesian_context_c123(line):
    listGroup = line[1]
    l = len(listGroup)  
    trainList = listGroup[0]      #0.8 train set
    testList = listGroup[1]       #0.2 test set
    trainList = [(t[0],t[1][4],t[1][5],t[1][6]) for t in trainList] #itemId,movingId
    testList = [(t[0],t[1][4],t[1][5],t[1][6]) for t in testList]   #itemId,movingID
    numTrain = len(trainList)
    numTest = l - numTrain

    newTestList = []
    for t in testList:
        context = [x for x in trainList if x[1]==t[1] and x[2]==t[2] and x[3]==t[3]]
        numContext = float(len(context))
        if numTrain != 0:
            p_context = numContext/numTrain  #P(C1i, C2j, C3k)
        else:
            p_context = 0
        p_app = [(-1,0),(-1,0),(-1,0),(-1,0),(-1,0)]
        context_no_duplicate = remove_duplicates(context)
        for c in context_no_duplicate:
            appi = [x for x in trainList if x[0]==c[0]]
            numAppi = float(len(appi))
            if numTrain != 0:
                p_appi = numAppi/numTrain
            else:
                p_appi = 0
            contextAppi = [x for x in trainList if x[0]==c[0] and x[1]==c[1] and x[2]==c[2] and x[3]==c[3]]
            if numAppi != 0:    #P(C1i, C2j, C3k | APPid)
                p_contextAppi = len(contextAppi)/numAppi 
            else:
                p_contextAppi = 0
            if p_context != 0:  #P(APPid | C1i,C2j,C3k = P(C1i, C2j, C3k | APPid)  P(APPid) /P(C1i, C2j, C3k)
                p = p_contextAppi * p_appi / p_context
            else:
                p = 0
            p_app = topFiveSortedList(p_app,c[0],p)
        p_app = sorted(p_app,key=lambda x: -x[1])
        app_rec = map(lambda x:x[0],p_app[:5])
        newTestList.append((t[0],app_rec))
    testList = [x[0] for x in newTestList]
    finalRecommender = [x[1] for x in newTestList]
    scores_rr = evaluation_rr(testList, finalRecommender)
    scores_ap = evaluation_ap(testList, finalRecommender)
    scores_recall = evaluation_recall(testList, finalRecommender)
    return scores_rr, scores_ap, scores_recall
finalScore = splitedRdd.map(bayesian_context_c123)
print_result(finalScore, 'bayesian_context_c123')

In [41]:
def topFiveSortedList(oldList, context, probability):
    context_oldList = [x[0] for x in oldList]
    if context not in context_oldList:
        for x in range(0,len(oldList)-1):
            if x == 0 and oldList[x][1] < probability:
                oldList[x] = (context,probability)
                if oldList[x+1][1] < oldList[x][1]:
                    temp = oldList[x+1] 
                    oldList[x+1] = oldList[x]
                    oldList[x] = temp 
                else:
                    break
            elif oldList[x+1][1] < oldList[x][1]:
                temp = oldList[x+1] 
                oldList[x+1] = oldList[x]
                oldList[x] = temp 
            else:
                break
    else:
        index = context_oldList.index(context)
        if oldList[index][1] < probability:
            oldList[index] = (context,probability)
        oldList = sorted(oldList,key=lambda x: x[1])
    return oldList
    #return index 

a = [(1,0.1),(2,0.2),(3,0.3),(4,0.4),(5,0.5)]
topFiveSortedList(a,3,0.7)

[(1, 0.1), (2, 0.2), (4, 0.4), (5, 0.5), (3, 0.7)]